In [3]:
import ee
import geemap as emap

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWhssDFpxKjVDdRLMItK_YJ06RqLkEPcPnXMCmO8AiYp3oXbFeNOGSA

Successfully saved authorization token.


- **Please see this technical guide of [eas website](https://sentinel.esa.int/web/sentinel/technical-guides/sentinel-2-msi/level-1c/cloud-masks) for the parameters of all Sentinel-2 images in GEE**

In [83]:
Map=emap.Map(center=[12,109],zoom=8)
Map.add_basemap("SATELLITE")
Map

Map(center=[12, 109], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

## Task 1: Monitoring abnomal vegetation using MODIS 

- **EVI Baseline 2000-2010**

In [84]:
# Select EVI MODIS from 2000 to 2010 and called reference data
collection = ee.ImageCollection('MODIS/006/MOD13A1').select('EVI')

reference = collection.filterDate('2001-01-01', '2010-12-31')\
  .sort('system:time_start')

- **EVI Recent Time 2011-2020**

In [85]:
# Get all MODIS EVI from 2011 to 2020 called recent
recent = collection.filterDate('2011-01-01', '2018-12-31')

Calculate the abnomal vegetation by subtracting median baseline from more recent images.

In [86]:
# Write a function to subtract the median baseline
def subtract(image):
    abnomal=image.multiply(0.0001).subtract(median_baseline)
    return abnomal.copyProperties(image,["stystem:time_start"])

In [90]:
# Map the function over the collection
abnomal_image=recent.map(subtract)

In [91]:
# Display the map
Map.addLayer(abnomal_image.sum(), {
  min: -10,
  max: 10,
  "palette": ['darkred','red','yellow','green','darkgreen']}, 'Cumulative anomaly');

Map

Map(bottom=14538.0, center=[22.938159639316396, 104.65576171875], controls=(WidgetControl(options=['position',…

### Calculate accumulative EVI through time-series 

In [223]:
time0=reference.first().get('system:time_start')

In [224]:
first = ee.List([ee.Image(0).set('system:time_start', time0).rename('EVI')])

In [225]:
def accumulate(image,mlist):
    previous=ee.Image(ee.List(mlist).get(-1))
    added=image.add(previous).set('system:time_start', image.get('system:time_start'))
    newList=ee.List(mlist).add(added)
    return newList

In [226]:
cumulative = ee.ImageCollection(ee.List(abnomal_image.iterate(accumulate, first)))